In [1]:


from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


1. Training Code for Model 1 (LSTM)


In [16]:
import librosa
import numpy as np
import tensorflow as tf
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, StandardScaler
from tensorflow.keras import layers, models
import os
import pandas as pd
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
import joblib
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

# Function to extract features from audio files
def extract_features(file_path):
    y, sr = librosa.load(file_path, duration=3)
    mfccs = librosa.feature.mfcc(y=y, sr=sr, n_mfcc=13)
    mel = librosa.feature.melspectrogram(y=y, sr=sr)
    contrast = librosa.feature.spectral_contrast(y=y, sr=sr)
    bandwidth = librosa.feature.spectral_bandwidth(y=y, sr=sr)
    energy_contour = librosa.feature.rms(y=y)
    spectral_rolloff = librosa.feature.spectral_rolloff(y=y, sr=sr)

    features = np.concatenate((
        np.mean(mfccs, axis=1),
        np.mean(mel, axis=1),
        np.mean(contrast, axis=1),
        np.mean(bandwidth, axis=1),
        np.mean(energy_contour, axis=1),
        np.mean(spectral_rolloff, axis=1)
    ))
    return features

# Folder containing audio files
audio_folder = '/content/drive/MyDrive/work2_extra/wav'

# List to store audio file paths and sentiment labels
file_names = []
senti_values = []

# Iterate through files in the folder
for file_name in os.listdir(audio_folder):
    if os.path.isfile(os.path.join(audio_folder, file_name)):
        file_names.append(os.path.join(audio_folder, file_name))

# Load sentiment labels from a CSV file
csv_path = '/content/drive/MyDrive/work2/Meta_Data/Audio.csv'
df = pd.read_csv(csv_path)
senti_values = df['Senti'].tolist()

# Create a dataset (X, y) with features and labels
X = np.array([extract_features(path) for path in file_names])
y = np.array(senti_values)

# Encode labels
label_encoder = LabelEncoder()
y = label_encoder.fit_transform(y)

# Check label distribution
unique, counts = np.unique(y, return_counts=True)
print("Label distribution:", dict(zip(unique, counts)))

# Split the dataset into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42, stratify=y)

# Standardize the features
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

# Save the scaler for later use
scaler_path = '/content/drive/MyDrive/work2/Final/Old_audio_models/scaler_lstm.pkl'
joblib.dump(scaler, scaler_path)

# Reshape for LSTM input
X_train = np.expand_dims(X_train, -1)
X_test = np.expand_dims(X_test, -1)

# Define early stopping and model checkpoint callbacks
early_stopping = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)
checkpoint = ModelCheckpoint('/content/drive/MyDrive/work2/Final/Old_audio_models/Audio_Model_1_LSTM.h5',
                             monitor='val_accuracy', save_best_only=True, mode='max')

# Build a neural network model with LSTM layers
model = models.Sequential()
model.add(layers.LSTM(128, return_sequences=True, input_shape=(X_train.shape[1], 1)))
model.add(layers.LSTM(64))
model.add(layers.Dense(128, activation='relu'))
model.add(layers.Dropout(0.5))
model.add(layers.Dense(len(np.unique(y)), activation='softmax'))

# Compile the model
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# Train the model with callbacks
history = model.fit(X_train, y_train, epochs=100, batch_size=64,
                    validation_data=(X_test, y_test),
                    callbacks=[early_stopping, checkpoint])

# Evaluate the best model on the test set
best_model = tf.keras.models.load_model('/content/drive/MyDrive/work2/Final/Old_audio_models/Audio_Model_1_LSTM.h5')
y_pred = best_model.predict(X_test)
y_pred_classes = np.argmax(y_pred, axis=1)
accuracy = accuracy_score(y_test, y_pred_classes)
precision = precision_score(y_test, y_pred_classes, average='weighted', zero_division=1)
recall = recall_score(y_test, y_pred_classes, average='weighted')
f1 = f1_score(y_test, y_pred_classes, average='weighted')

# Print evaluation metrics
print(f"Accuracy: {accuracy*100:.2f}%")
print(f"Precision: {precision:.2f}")
print(f"Recall: {recall:.2f}")
print(f"F1 Score: {f1:.2f}")

# Save results in a pandas DataFrame
results_df = pd.DataFrame({
    'Model Name': ['LSTM'],
    'Model Path': ['/content/drive/MyDrive/work2/Final/Old_audio_models/Audio_Model_1_LSTM.h5'],
    'Accuracy': [accuracy],
    'Precision': [precision],
    'Recall': [recall],
    'F1 Score': [f1]
})

# Path to the results file
results_file = '/content/drive/MyDrive/work2/Final/new_evaluation_results.csv'

# Check if the results file exists
if os.path.exists(results_file):
    # If it exists, read the existing data
    existing_df = pd.read_csv(results_file)
    # Append the new results
    results_df = pd.concat([existing_df, results_df], ignore_index=True)

# Save the DataFrame to the CSV file
results_df.to_csv(results_file, index=False)

print(f"Results saved to {results_file}")


Label distribution: {0: 17, 1: 54, 2: 40}
Epoch 1/100
2/2 [==============================] - 7s 1s/step - loss: 1.0960 - accuracy: 0.3896 - val_loss: 1.0834 - val_accuracy: 0.5000
Epoch 2/100


/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


2/2 [==============================] - 1s 336ms/step - loss: 1.0808 - accuracy: 0.4935 - val_loss: 1.0718 - val_accuracy: 0.5000
Epoch 3/100
2/2 [==============================] - 1s 340ms/step - loss: 1.0704 - accuracy: 0.4935 - val_loss: 1.0593 - val_accuracy: 0.5000
Epoch 4/100
2/2 [==============================] - 1s 283ms/step - loss: 1.0575 - accuracy: 0.5195 - val_loss: 1.0436 - val_accuracy: 0.5000
Epoch 5/100
2/2 [==============================] - 1s 432ms/step - loss: 1.0354 - accuracy: 0.5325 - val_loss: 1.0228 - val_accuracy: 0.5294
Epoch 6/100
2/2 [==============================] - 1s 335ms/step - loss: 1.0136 - accuracy: 0.5325 - val_loss: 1.0103 - val_accuracy: 0.5000
Epoch 7/100
2/2 [==============================] - 1s 348ms/step - loss: 0.9941 - accuracy: 0.5195 - val_loss: 1.0569 - val_accuracy: 0.4706
Epoch 8/100
2/2 [==============================] - 1s 346ms/step - loss: 0.9873 - accuracy: 0.5325 - val_loss: 1.0794 - val_accuracy: 0.4706
Epoch 9/100
2/2 [========

2/2 [==============================] - 1s 34ms/step
Accuracy: 52.94%
Precision: 0.59
Recall: 0.53
F1 Score: 0.40
Results saved to /content/drive/MyDrive/work2/Final/new_evaluation_results.csv


In [14]:
import librosa
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
import os
import pandas as pd
import joblib

# Function to extract features from audio files
def extract_features(file_path):
    y, sr = librosa.load(file_path, duration=3)
    mfccs = librosa.feature.mfcc(y=y, sr=sr, n_mfcc=13)
    mel = librosa.feature.melspectrogram(y=y, sr=sr)
    contrast = librosa.feature.spectral_contrast(y=y, sr=sr)
    bandwidth = librosa.feature.spectral_bandwidth(y=y, sr=sr)
    energy_contour = librosa.feature.rms(y=y)
    spectral_rolloff = librosa.feature.spectral_rolloff(y=y, sr=sr)

    features = np.concatenate((
        np.mean(mfccs, axis=1),
        np.mean(mel, axis=1),
        np.mean(contrast, axis=1),
        np.mean(bandwidth, axis=1),
        np.mean(energy_contour, axis=1),
        np.mean(spectral_rolloff, axis=1)
    ))
    return features

# Folder containing audio files
audio_folder = '/content/drive/MyDrive/work2_extra/wav'

# List to store audio file paths and sentiment labels
file_names = []
senti_values = []

# Iterate through files in the folder
for file_name in os.listdir(audio_folder):
    if os.path.isfile(os.path.join(audio_folder, file_name)):
        file_names.append(os.path.join(audio_folder, file_name))

# Load sentiment labels from a CSV file
csv_path = '/content/drive/MyDrive/work2/Meta_Data/Audio.csv'
df = pd.read_csv(csv_path)
senti_values = df['Senti'].tolist()

# Create a dataset (X, y) with features and labels
X = np.array([extract_features(path) for path in file_names])
y = np.array(senti_values)

# Print some feature vectors to verify
print("Sample features:", X[:5])

# Encode labels
label_encoder = LabelEncoder()
y = label_encoder.fit_transform(y)

# Check label distribution
unique, counts = np.unique(y, return_counts=True)
print("Label distribution:", dict(zip(unique, counts)))

# Split the dataset into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

# Standardize the features
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

# Save the scaler for later use
scaler_path = '/content/drive/MyDrive/work2/Final/Old_audio_models/scaler_rf.pkl'
joblib.dump(scaler, scaler_path)

# Train a Random Forest classifier
model = RandomForestClassifier(n_estimators=200, random_state=42, max_depth=30)
model.fit(X_train, y_train)

# Evaluate the model on the test set
y_pred = model.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
precision = precision_score(y_test, y_pred, average='weighted', zero_division=1)
recall = recall_score(y_test, y_pred, average='weighted')
f1 = f1_score(y_test, y_pred, average='weighted')

# Print evaluation metrics
print(f"Accuracy: {accuracy*100:.2f}%")
print(f"Precision: {precision:.2f}")
print(f"Recall: {recall:.2f}")
print(f"F1 Score: {f1:.2f}")

# Save results in a pandas DataFrame
results_df = pd.DataFrame({
    'Model Name': ['Random Forest'],
    'Model Path': ['/content/drive/MyDrive/work2/Final/Old_audio_models/Audio_Model_2_RandomForest.joblib'],
    'Accuracy': [accuracy],
    'Precision': [precision],
    'Recall': [recall],
    'F1 Score': [f1]
})

# Path to the results file
results_file = '/content/drive/MyDrive/work2/Final/new_evaluation_results.csv'

# Check if the results file exists
if os.path.exists(results_file):
    # If it exists, read the existing data
    existing_df = pd.read_csv(results_file)
    # Append the new results
    results_df = pd.concat([existing_df, results_df], ignore_index=True)

# Save the DataFrame to the CSV file
results_df.to_csv(results_file, index=False)

print(f"Results saved to {results_file}")


Sample features: [[-4.52683594e+02  2.01943684e+01 -1.51375227e+01  2.85106812e+01
  -2.77991505e+01  8.19266796e+00 -4.41237354e+00 -6.33207703e+00
  -1.13443794e+01 -1.01665754e+01 -1.21117163e+01 -1.81381559e+00
  -1.16885214e+01  1.87976530e-05  4.02092992e-05  2.66110030e-04
   3.93119291e-04  3.25598492e-04  6.71495858e-04  1.99865969e-03
   2.04756409e-02  5.92655480e-01  1.31503296e+00  1.80886781e+00
   4.19252247e-01  2.74035819e-02  4.35651839e-03  2.56317807e-03
   7.63226242e-04  1.11585355e-03  1.86926350e-02  1.76759794e-01
   2.57734984e-01  4.57792968e-01  1.35186657e-01  4.32332465e-03
   7.60768657e-04  4.06371953e-04  3.86215688e-04  5.58496511e-04
   3.41302739e-03  3.34950816e-03  6.35737833e-03  8.24973285e-02
   1.27190307e-01  1.69017352e-02  1.19511562e-03  7.02867401e-04
   1.56437210e-03  1.10956989e-02  1.06652146e-02  5.20917540e-03
   3.00668320e-03  2.42063981e-02  4.68492433e-02  1.11661013e-02
   2.67059216e-03  2.99850339e-03  8.57532118e-03  9.302014

In [17]:
import pandas as pd
results_file = '/content/drive/MyDrive/work2/Final/new_evaluation_results.csv'
df = pd.read_csv(results_file)
df.head(10)



,Model Name,Model Path,Accuracy,Precision,Recall,F1 Score
0,LSTM,/content/drive/MyDrive/work2/Final/Old_audio_m...,0.478261,0.398551,0.478261,0.376878
1,Random Forest,/content/drive/MyDrive/work2/Final/Old_audio_m...,0.478261,0.465675,0.478261,0.414493
2,LSTM,/content/drive/MyDrive/work2/Final/Old_audio_m...,0.478261,0.466874,0.478261,0.372492
3,Random Forest,/content/drive/MyDrive/work2/Final/Old_audio_m...,0.478261,0.465675,0.478261,0.414493
4,LSTM,/content/drive/MyDrive/work2/Final/Old_audio_m...,0.478261,0.528986,0.478261,0.371797
5,Random Forest,/content/drive/MyDrive/work2/Final/Old_audio_m...,0.347826,0.432842,0.347826,0.294364
6,LSTM,/content/drive/MyDrive/work2/Final/Old_audio_m...,0.434783,0.482609,0.434783,0.350363
7,LSTM,/content/drive/MyDrive/work2/Final/Old_audio_m...,0.478261,0.528986,0.478261,0.371797
8,Random Forest,/content/drive/MyDrive/work2/Final/Old_audio_m...,0.347826,0.432842,0.347826,0.294364
9,LSTM,/content/drive/MyDrive/work2/Final/Old_audio_m...,0.529412,0.589154,0.529412,0.397359
